In [ ]:
#https://www.sitepoint.com/keras-digit-recognition-tutorial/

Importing Dataset

In [ ]:
import numpy as np

def create_scaler(*args):
  min_d = np.min(args[0])
  max_d = np.max(args[0])
  for partition in args:
    if np.min(partition) < min_d:
      min_d = np.min(partition)
    if np.max(partition) > max_d:
      max_d = np.max(partition)
  return [min_d, max_d]

def normalizer(scaler, df):
  min_d = scaler[0]
  max_d = scaler[1]
  normalized_df = (df - min_d) / (max_d - min_d)
  return normalized_df

def denormalizer(scaler, normalized_df):
  min_d = scaler[0]
  max_d = scaler[1]
  denormalized_df = normalized_df * (max_d - min_d) + min_d
  return denormalized_df

In [ ]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

Preprocessing

In [ ]:
#show one image with label
print(y_train[0])
plt.imshow(X_train[0], cmap='Greys')
plt.show()

In [ ]:
#change shape to IMAGES only for CNN input
X_train = X_train.reshape(60000, 28, 28, 1)
X_test = X_test.reshape(10000, 28, 28, 1)
#the labels are already in an acceptable shape

In [ ]:
#normalize image data only
scaler = create_scaler([0, 255])
X_train_ = normalizer(scaler, X_train)
X_test_ = normalizer(scaler, X_test)

In [ ]:
#DEPRECATED: normalize, identical to the new method
import numpy as np
X_train = np.array(X_train, dtype=np.float64)
X_test = np.array(X_test, dtype=np.float64)
X_train /= 255
X_test /= 255

In [ ]:
'''
#se lo si attiva, il modello si sputtana
from keras.utils import to_categorical
num_classes = 10

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
#y sono i labels one_hot: [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]
'''

CNN

In [ ]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
     activation='relu',
     input_shape=(28, 28, 1))) #image size
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax')) #number of classes

model.compile(loss='sparse_categorical_crossentropy',
      optimizer='adam',
      metrics=['accuracy'])

batch_size = 128
epochs = 10

model.fit(X_train_, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#   summarize model_1
#input layer not included
model.summary(line_length=None, positions=None, print_fn=None)

In [ ]:
#   summarize model_2
#input layer is included
from keras.utils.vis_utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True)

Image

In [ ]:
import imageio
import numpy as np
from matplotlib import pyplot as plt

im = imageio.imread("https://i.imgur.com/a3Rql9C.png")

In [ ]:
#visualize image
gray = np.dot(im[...,:3], [0.299, 0.587, 0.114])
plt.imshow(gray, cmap = plt.get_cmap('gray'))
plt.show()

In [ ]:
# reshape the image
gray = gray.reshape(1, 28, 28, 1)

# normalize image
gray /= 255

Prediction

In [ ]:
#predict digit in x_train
prediction = model.predict(X_test[0].reshape(1, 28, 28, 1))
print(y_test[0], prediction.argmax())

In [ ]:
print(y_test[0])
plt.imshow(X_test[0], cmap='Greys')
plt.show()

In [ ]:
#predict digit from img
prediction = model.predict(gray)
print(prediction.argmax())